In [8]:
%run task_setup.ipynb

Foundation: 2.11.030
Transition: 3.02.029


In [9]:
# Build status
has_contract = False
default_save = True

------------------
-----------------
# Domain Contract Initialisation

--------------------
##  Transaction setup

In [10]:
for task in ['members', 'cln_members', 'ins_members', 'prf_members', 'flu_risk']:
    project = 'datalake_gen' if task == 'flu_risk' else 'factory'
    version = 'v06' if task == 'flu_risk' else 'v05'
    # setup the source inputs
    tr = Transition.from_env(task, default_save=default_save, has_contract=has_contract)
    tr.set_version(VERSION)
    tr.set_source_uri(uri=f"s3://project-hadron-cs-repo/domain/helloworld/data/datalake_sim/members/{project}_transition_{task}_dataset_{version}.parquet")
    tr.setup_bootstrap(domain=domain, project_name=project_name, path=None)
    tr.set_provenance(provider_name="System of Records")

--------------------
##  Wrangle setup

In [11]:
wr = Wrangle.from_env('profile', default_save=default_save, has_contract=has_contract)

In [12]:
wr.set_version(VERSION)
wr.setup_bootstrap(domain=domain, project_name=project_name, path=None)
# set the source as the members dataset
wr.set_source_uri(uri=Transition.from_env('members').get_persist_contract().raw_uri, template_aligned=True)

# set the other connector 
for task in ['cln_members', 'ins_members', 'prf_members', 'flu_risk']:
    wr.add_connector_uri(connector_name=task, uri=Transition.from_env(task).get_persist_contract().raw_uri, template_aligned=True)


--------------------
##  Transaction setup

In [13]:
tr = Transition.from_env('profile', default_save=default_save, has_contract=has_contract)

In [14]:
tr.set_version(VERSION)
tr.setup_bootstrap(domain=domain, project_name=project_name, path=None)
tr.set_provenance(provider_name="Profile AI", author_name='doatridge')
tr.set_source_uri(uri=wr.from_env('profile').get_persist_contract().raw_uri, template_aligned=True)
